Can we create a carbon copy of a boundary condition file? We'll copy over the actual BC data from an example file, but everything else will be built from scratch.

In [1]:
import h5py
from netCDF4 import Dataset
import numpy as np
from pathlib import Path

import xarray as xr

Unable to import google auth packages


# For reference, look at a stereotypical boundary condition file

In [2]:
### Open and characterize a stereotypical group
# template_file = Path('/projects/wakedynamics/orybchuk/bcs-ldm/simulations/testing_amrwind/precursor_for_quiescent_bcs/bndry_file.nc')
template_file = Path('/scratch/orybchuk/share/wakedynamics/bcs-ldm/Q3/template_bndry_file.nc')
with Dataset(template_file, "r") as template:
    print("template.cmptypes", template.cmptypes)
    print("template.data_model", template.data_model)
    print("template.dimensions", template.dimensions)
    print("template.disk_format", template.disk_format)
    print("template.enumtypes", template.enumtypes)
    print("template.file_format", template.file_format)
    print("template.groups", template.groups)
    print("template.keepweakref", template.keepweakref)
    print("template.name", template.name)
    print("template.parent", template.parent)
    print("template.path", template.path)
    print("template.variables", template.variables)
    print("template.vltypes", template.vltypes)

template.cmptypes {}
template.data_model NETCDF4
template.dimensions {'sdim': <class 'netCDF4._netCDF4.Dimension'>: name = 'sdim', size = 1, 'pdim': <class 'netCDF4._netCDF4.Dimension'>: name = 'pdim', size = 2, 'vdim': <class 'netCDF4._netCDF4.Dimension'>: name = 'vdim', size = 3, 'nt': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'nt', size = 1803}
template.disk_format HDF5
template.enumtypes {}
template.file_format NETCDF4
template.groups {'xlo': <class 'netCDF4._netCDF4.Group'>
group /xlo:
    dimensions(sizes): 
    variables(dimensions): int32 normal(), int32 side(), int32 perpendicular(pdim)
    groups: level_0}
template.keepweakref False
template.name /
template.parent None
template.path /
template.variables {'time': <class 'netCDF4._netCDF4.Variable'>
float64 time(nt)
unlimited dimensions: nt
current shape = (1803,)
filling on, default _FillValue of 9.969209968386869e+36 used}
template.vltypes {}


### Look at the stuff I'll need to add in greater detail

In [3]:
### Look at the dimensions
with Dataset(template_file, "r") as template:
    print(template.dimensions)
    print()
    print(type(template.dimensions))
    print()
    print(template.dimensions['sdim'])
    print(template.dimensions['pdim'])
    print(template.dimensions['vdim'])
    print(template.dimensions['nt'])
    
    print(len(template.dimensions['pdim']))

{'sdim': <class 'netCDF4._netCDF4.Dimension'>: name = 'sdim', size = 1, 'pdim': <class 'netCDF4._netCDF4.Dimension'>: name = 'pdim', size = 2, 'vdim': <class 'netCDF4._netCDF4.Dimension'>: name = 'vdim', size = 3, 'nt': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'nt', size = 1803}

<class 'dict'>

<class 'netCDF4._netCDF4.Dimension'>: name = 'sdim', size = 1
<class 'netCDF4._netCDF4.Dimension'>: name = 'pdim', size = 2
<class 'netCDF4._netCDF4.Dimension'>: name = 'vdim', size = 3
<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'nt', size = 1803
2


In [4]:
### Look at the variables in the root group
with Dataset(template_file, "r") as template:
    print(template.variables)

{'time': <class 'netCDF4._netCDF4.Variable'>
float64 time(nt)
unlimited dimensions: nt
current shape = (1803,)
filling on, default _FillValue of 9.969209968386869e+36 used}


In [5]:
### Look at the xlo/ylo subgroups in detail
with Dataset(template_file, "r") as template:
    # print(template['xlo'].dimensions)
    # print()
    # print(template['xlo'].groups)
    # print()
    print(template['xlo'].variables)
    print()
    print(template['xlo']['normal'][:])
    print(template['xlo']['side'][:])
    print(template['xlo']['perpendicular'][:])

{'normal': <class 'netCDF4._netCDF4.Variable'>
int32 normal()
path = /xlo
unlimited dimensions: 
current shape = ()
filling on, default _FillValue of -2147483647 used, 'side': <class 'netCDF4._netCDF4.Variable'>
int32 side()
path = /xlo
unlimited dimensions: 
current shape = ()
filling on, default _FillValue of -2147483647 used, 'perpendicular': <class 'netCDF4._netCDF4.Variable'>
int32 perpendicular(pdim)
path = /xlo
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of -2147483647 used}

0
0
[1 2]


In [6]:
### Look at the xlo/level_0 subgroups in detail
with Dataset(template_file, "r") as template:
    # print(template['xlo/level_0'].dimensions)
    # print(template['xlo/level_0'].dimensions['nx'])
    # print()
    # print(template['xlo/level_0'].groups)
    # print()
    print(template['xlo/level_0'].variables)
    print()
    print(template['xlo/level_0'].variables['lengths'][:])
    print(template['xlo/level_0'].variables['lo'][:])
    print(template['xlo/level_0'].variables['hi'][:])
    print(template['xlo/level_0'].variables['dx'][:])
    print(template['xlo/level_0'].variables['velocity'].shape)
    print(template['xlo/level_0'].variables['temperature'].shape)
    
    xlo_velocity_template = template['xlo/level_0'].variables['velocity'][:]
    xlo_temperature_template = template['xlo/level_0'].variables['temperature'][:]

    print(template['xlo']['normal'][:])
    print(template['xlo']['side'][:])
    print(template['xlo']['perpendicular'][:])

{'lengths': <class 'netCDF4._netCDF4.Variable'>
float64 lengths(pdim)
path = /xlo/level_0
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'lo': <class 'netCDF4._netCDF4.Variable'>
float64 lo(pdim)
path = /xlo/level_0
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'hi': <class 'netCDF4._netCDF4.Variable'>
float64 hi(pdim)
path = /xlo/level_0
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'dx': <class 'netCDF4._netCDF4.Variable'>
float64 dx(pdim)
path = /xlo/level_0
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'velocity': <class 'netCDF4._netCDF4.Variable'>
float64 velocity(nt, ny, nz, vdim)
path = /xlo/level_0
unlimited dimensions: nt
current shape = (1803, 256, 128, 3)
filling on, default _FillValue of 9.969209968386869e+36 used, 'temperature': <class '

In [7]:
xlo_velocity_template.shape

(1803, 256, 128, 3)

In [8]:
xlo_temperature_template.shape

(1803, 256, 128)

# Create a boundary condition file

In [9]:
### User inputs
# parent_dir = Path('/scratch/orybchuk/wakedynamics/bcs-ldm/palette_3d/concat_and_cascade/2023_06_15_18_27_chunked0001/12_to_24')
parent_dir = Path('/scratch/orybchuk/share/wakedynamics/bcs-ldm/Q3')
data_file = Path(parent_dir, 'concatenated_inflow_bc.nc')
outfile = Path(parent_dir, 'amrwind_inflow_bc.nc')

# TODO: MATCH THESE VALUES TO THE SIMULATION
nx = 9999
ny = 32
nz = 24
nfields = 5

dx, dy, dz = 10., 10., 10.
Ly, Lz = ny*dy, nz*dz
ylo, zlo = 0., 0.
yhi, zhi = Ly, Lz

dt = 1.0

In [10]:
### Extract data from netCDF and reformat as numpy
ds_bc = xr.open_dataset(data_file)

ntime = len(ds_bc['time'])

velocity_bc = np.zeros((ntime, ny, nz, 3))
velocity_bc[:,:,:,0] = ds_bc['u_output'].values
velocity_bc[:,:,:,1] = ds_bc['v_output'].values
velocity_bc[:,:,:,2] = ds_bc['w_output'].values
temperature_bc = ds_bc['T_output'].values
tke_bc = ds_bc['TKE_output'].values

time_bc = np.arange(0,ntime,dt)
time_bc = np.ma.masked_array(data=time_bc, mask=False, fill_value=1e20)

In [11]:
# outfile = Path(template_file.parent, synthetic_bcs.nc)
with Dataset(outfile, "w", format="NETCDF4") as rootgrp:
    ## ~~~~~~~~ Operate on the root group ~~~~~~~~
    # Add dimensions to the root group
    rootgrp.createDimension('sdim', 1)
    rootgrp.createDimension('pdim', 2)
    rootgrp.createDimension('vdim', 3)
    rootgrp.createDimension('nt', None)
    
    # Add groups
    grp_xlo = rootgrp.createGroup("/xlo")
    
    # Add variables    
    rootgrp.createVariable("time","f8",("nt",))
    rootgrp['time'][:] = time_bc

    
    ## ~~~~~~~~ Operate on the xlo and ylo groups ~~~~~~~~
    # No need to add dimensions
    
    # Add groups
    grp_xlo_l0 = grp_xlo.createGroup('/level_0')

    # Add variables
    grp_xlo.createVariable("normal","i4",)
    grp_xlo.createVariable("side","i4",)
    grp_xlo.createVariable("perpendicular","i4",("pdim",))
    
    grp_xlo['normal'][:] = 0
    grp_xlo['side'][:] = 0
    grp_xlo['perpendicular'][:] = [1, 2]

    ## ~~~~~~~~ Operate on the xlo/level_0 and ylo/level_0 groups ~~~~~~~~
    # Add dimensions
    grp_xlo_l0.createDimension('nx', nx)
    grp_xlo_l0.createDimension('ny', ny)
    grp_xlo_l0.createDimension('nz', nz)

    # No need to add groups
    
    # Add variables
    grp_xlo_l0.createVariable("lengths","f8",("pdim",))
    grp_xlo_l0.createVariable("lo","f8",("pdim",))
    grp_xlo_l0.createVariable("hi","f8",("pdim",))
    grp_xlo_l0.createVariable("dx","f8",("pdim",))
    grp_xlo_l0.createVariable("velocity","f8",("nt", "ny", "nz", "vdim",))
    grp_xlo_l0.createVariable("temperature","f8",("nt", "ny", "nz",))
    grp_xlo_l0.createVariable("tke","f8",("nt", "ny", "nz",))  # TODO: Am I precribing SGS TKE correctly?
    
    grp_xlo_l0['lengths'][:] = [Ly, Lz]
    grp_xlo_l0['lo'][:] = [ylo, zlo]
    grp_xlo_l0['hi'][:] = [yhi, zhi]
    grp_xlo_l0['dx'][:] = [dy, dz]
    grp_xlo_l0['velocity'][:] = velocity_bc
    grp_xlo_l0['temperature'][:] = temperature_bc
    grp_xlo_l0['tke'][:] = tke_bc

# Do a line-by-line comparison between what we're supposed to look like and what we actually look like

In [12]:
## Compare at the root-group level
with Dataset(template_file, "r") as template:
    print("template.cmptypes", template.cmptypes)
    print("template.data_model", template.data_model)
    print("template.dimensions", template.dimensions)
    print("template.disk_format", template.disk_format)
    print("template.enumtypes", template.enumtypes)
    print("template.file_format", template.file_format)
    print("template.groups", template.groups)
    print("template.keepweakref", template.keepweakref)
    print("template.name", template.name)
    print("template.parent", template.parent)
    print("template.path", template.path)
    print("template.variables", template.variables)
    print("template.vltypes", template.vltypes)
    print("template['time'][:]", template['time'][:])
    
print()
    
with Dataset(outfile, "r") as rootgrp:
    print("rootgrp.cmptypes", rootgrp.cmptypes)
    print("rootgrp.data_model", rootgrp.data_model)
    print("rootgrp.dimensions", rootgrp.dimensions)
    print("rootgrp.disk_format", rootgrp.disk_format)
    print("rootgrp.enumtypes", rootgrp.enumtypes)
    print("rootgrp.file_format", rootgrp.file_format)
    print("rootgrp.groups", rootgrp.groups)
    print("rootgrp.keepweakref", rootgrp.keepweakref)
    print("rootgrp.name", rootgrp.name)
    print("rootgrp.parent", rootgrp.parent)
    print("rootgrp.path", rootgrp.path)
    print("rootgrp.variables", rootgrp.variables)
    print("rootgrp.vltypes", rootgrp.vltypes)
    print("rootgrp['time'][:]", rootgrp['time'][:])

template.cmptypes {}
template.data_model NETCDF4
template.dimensions {'sdim': <class 'netCDF4._netCDF4.Dimension'>: name = 'sdim', size = 1, 'pdim': <class 'netCDF4._netCDF4.Dimension'>: name = 'pdim', size = 2, 'vdim': <class 'netCDF4._netCDF4.Dimension'>: name = 'vdim', size = 3, 'nt': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'nt', size = 1803}
template.disk_format HDF5
template.enumtypes {}
template.file_format NETCDF4
template.groups {'xlo': <class 'netCDF4._netCDF4.Group'>
group /xlo:
    dimensions(sizes): 
    variables(dimensions): int32 normal(), int32 side(), int32 perpendicular(pdim)
    groups: level_0}
template.keepweakref False
template.name /
template.parent None
template.path /
template.variables {'time': <class 'netCDF4._netCDF4.Variable'>
float64 time(nt)
unlimited dimensions: nt
current shape = (1803,)
filling on, default _FillValue of 9.969209968386869e+36 used}
template.vltypes {}
template['time'][:] [10800.  10800.5 10801.  ... 11700.  11700.5 1170

In [13]:
## Compare at the xlo-group level
with Dataset(template_file, "r") as template:
    print("template['xlo']", template['xlo'])
    print("template['xlo'].dimensions", template['xlo'].dimensions)
    print("template['xlo'].variables", template['xlo'].variables)
    print("template['xlo']['normal'][:]", template['xlo']['normal'][:])
    print("template['xlo']['side'][:]", template['xlo']['side'][:])
    print("template['xlo']['perpendicular'][:]", template['xlo']['perpendicular'][:])
    print("template['xlo'].groups", template['xlo'].groups)
    
print()
    
with Dataset(outfile, "r") as rootgrp:
    print("rootgrp['xlo']", rootgrp['xlo'])
    print("rootgrp['xlo'].dimensions", rootgrp['xlo'].dimensions)
    print("rootgrp['xlo'].variables", rootgrp['xlo'].variables)
    print("rootgrp['xlo']['normal'][:]", rootgrp['xlo']['normal'][:])
    print("rootgrp['xlo']['side'][:]", rootgrp['xlo']['side'][:])
    print("rootgrp['xlo']['perpendicular'][:]", rootgrp['xlo']['perpendicular'][:])
    print("rootgrp['xlo'].groups", rootgrp['xlo'].groups)

template['xlo'] <class 'netCDF4._netCDF4.Group'>
group /xlo:
    dimensions(sizes): 
    variables(dimensions): int32 normal(), int32 side(), int32 perpendicular(pdim)
    groups: level_0
template['xlo'].dimensions {}
template['xlo'].variables {'normal': <class 'netCDF4._netCDF4.Variable'>
int32 normal()
path = /xlo
unlimited dimensions: 
current shape = ()
filling on, default _FillValue of -2147483647 used, 'side': <class 'netCDF4._netCDF4.Variable'>
int32 side()
path = /xlo
unlimited dimensions: 
current shape = ()
filling on, default _FillValue of -2147483647 used, 'perpendicular': <class 'netCDF4._netCDF4.Variable'>
int32 perpendicular(pdim)
path = /xlo
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of -2147483647 used}
template['xlo']['normal'][:] 0
template['xlo']['side'][:] 0
template['xlo']['perpendicular'][:] [1 2]
template['xlo'].groups {'level_0': <class 'netCDF4._netCDF4.Group'>
group /xlo/level_0:
    dimensions(sizes): nx(256), ny(256), nz(128)

In [14]:
## Compare at the xlo-level0-group level
with Dataset(template_file, "r") as template:
    print("template['xlo']['level_0']", template['xlo']['level_0'])
    print("template['xlo']['level_0'].dimensions", template['xlo']['level_0'].dimensions)
    print("template['xlo']['level_0'].variables", template['xlo']['level_0'].variables)
    print("template['xlo']['level_0']['lengths'][:]", template['xlo']['level_0']['lengths'][:])
    print("template['xlo']['level_0']['lo'][:]", template['xlo']['level_0']['lo'][:])
    print("template['xlo']['level_0']['hi'][:]", template['xlo']['level_0']['hi'][:])
    print("template['xlo']['level_0']['dx'][:]", template['xlo']['level_0']['dx'][:])
    print("template['xlo']['level_0']['velocity'][-5:,0,0,:]", template['xlo']['level_0']['velocity'][-5:,0,0,:])
    print("template['xlo']['level_0']['temperature'][-5:,0,0]", template['xlo']['level_0']['temperature'][-5:,0,0])
    print("template['xlo'].groups", template['xlo'].groups)
    
print()
    
with Dataset(outfile, "r") as rootgrp:
    print("rootgrp['xlo']['level_0']", rootgrp['xlo']['level_0'])
    print("rootgrp['xlo']['level_0'].dimensions", rootgrp['xlo']['level_0'].dimensions)
    print("rootgrp['xlo']['level_0'].variables", rootgrp['xlo']['level_0'].variables)
    print("rootgrp['xlo']['level_0']['lengths'][:]", rootgrp['xlo']['level_0']['lengths'][:])
    print("rootgrp['xlo']['level_0']['lo'][:]", rootgrp['xlo']['level_0']['lo'][:])
    print("rootgrp['xlo']['level_0']['hi'][:]", rootgrp['xlo']['level_0']['hi'][:])
    print("rootgrp['xlo']['level_0']['dx'][:]", rootgrp['xlo']['level_0']['dx'][:])
    print("rootgrp['xlo']['level_0']['velocity'][-5:,0,0,:]", rootgrp['xlo']['level_0']['velocity'][-5:,0,0,:])
    print("rootgrp['xlo']['level_0']['temperature'][-5:,0,0]", rootgrp['xlo']['level_0']['temperature'][-5:,0,0])
    print("rootgrp['xlo'].groups", rootgrp['xlo'].groups)

template['xlo']['level_0'] <class 'netCDF4._netCDF4.Group'>
group /xlo/level_0:
    dimensions(sizes): nx(256), ny(256), nz(128)
    variables(dimensions): float64 lengths(pdim), float64 lo(pdim), float64 hi(pdim), float64 dx(pdim), float64 velocity(nt, ny, nz, vdim), float64 temperature(nt, ny, nz)
    groups: 
template['xlo']['level_0'].dimensions {'nx': <class 'netCDF4._netCDF4.Dimension'>: name = 'nx', size = 256, 'ny': <class 'netCDF4._netCDF4.Dimension'>: name = 'ny', size = 256, 'nz': <class 'netCDF4._netCDF4.Dimension'>: name = 'nz', size = 128}
template['xlo']['level_0'].variables {'lengths': <class 'netCDF4._netCDF4.Variable'>
float64 lengths(pdim)
path = /xlo/level_0
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'lo': <class 'netCDF4._netCDF4.Variable'>
float64 lo(pdim)
path = /xlo/level_0
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'hi': <class 'netCDF4.